## 벡터의 유사도(Vector Similarity)

### numpy 사용해서 코사인 유사도 계산

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print('문서1과 2의 유사도: ', cos_sim(doc1, doc2))
print('문서1과 3의 유사도: ', cos_sim(doc1, doc3))
print('문서2와 3의 유사도: ', cos_sim(doc2, doc3))

문서1과 2의 유사도:  0.6666666666666667
문서1과 3의 유사도:  0.6666666666666667
문서2와 3의 유사도:  1.0000000000000002


### 유사도를 이용한 추천시스템 구현
- title과 overview(줄거리)를 코사인 유사도에 사용
- 좋아하는 영화 입력하면 해당 영화의 줄거리와 유사한 줄거리의 영화 찾아서 추천

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
# 상위 2개만의 샘플을 data에 저장
data = data.head(20000)

In [4]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 결측값 수: ', data['overview'].isnull().sum())

overview 결측값 수:  135


In [5]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

In [6]:
# overview열에 대해 TF-IDF 구한 후 행렬의 크기 출력
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF의 크기(shape): ', tfidf_matrix.shape)

TF-IDF의 크기(shape):  (20000, 47487)


20,000개의 영화를 표현하기 위해 47,487개 단어가 사용되었음을 의미
- 47,487 차원의 문서 벡터가 20,000개 존재한다고도 표현 가능

In [7]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과: ', cosine_sim.shape)

코사인 유사도 연산 결과:  (20000, 20000)


모든 영화의 상호 유사도 기록

In [8]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [9]:
# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다
    idx = title_to_index[title]
    
    # 해당 영화와 모든 영화와의 유사도를 가져온다
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 영화 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 가장 유사한 10개의 영화 빋아옴
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개의 영화 인덱스
    movie_indices = [idx[0] for idx in sim_scores]
    
    # 가장 유사한 10개의 영화 제목 리턴
    return data['title'].iloc[movie_indices]

In [10]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

전부 배트맨 영화 찾아낸 것 확인

### 유클리디안 거리(Euclidean Distance)

In [11]:
import numpy as np

def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

# DTM
doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print('문서1과 문서Q의 거리 :',dist(doc1,docQ))
print('문서2과 문서Q의 거리 :',dist(doc2,docQ))
print('문서3과 문서Q의 거리 :',dist(doc3,docQ))

문서1과 문서Q의 거리 : 2.23606797749979
문서2과 문서Q의 거리 : 3.1622776601683795
문서3과 문서Q의 거리 : 2.449489742783178


### 자카드 유사도(Jaccard similarity)

In [12]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [13]:
union = set(tokenized_doc1).union(set(tokenized_doc2))
print('문서1과 문서2의 합집합 :',union)

문서1과 문서2의 합집합 : {'coupon', 'love', 'like', 'likey', 'you', 'watch', 'passport', 'holder', 'apple', 'everyone', 'banana', 'card'}


In [14]:
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print('문서1과 문서2의 교집합 :',intersection)

문서1과 문서2의 교집합 : {'banana', 'apple'}


In [15]:
print('자카드 유사도 :',len(intersection)/len(union))

자카드 유사도 : 0.16666666666666666
